In [0]:
%pip install scikit-image==0.20.0

In [0]:
import geopandas as gpd
import shutil
import os
from pyspark.sql.functions import *

from src.data.predict_utils import get_kommune_and_fylke

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
gold_table = f"{catalog_dev}.{schema_dev}.predicted_helipads_gold"

In [0]:
catalov_eiendom = "eiendom_smia_dev"
schema_eiendom = "admin_enheter_silver"
kommune_table = f"{catalov_eiendom}.{schema_eiendom}.kommune"
fylke_table = f"{catalov_eiendom}.{schema_eiendom}.fylke"
srid = 25833

In [0]:
local_gpkg = os.path.join(os.getcwd(), "predicted_helipads.gpkg")
final_gpkg_dir = "/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/geopackages/"
final_gpkg_path = os.path.join(final_gpkg_dir, "predicted_helipads.gpkg")

joined_df = get_kommune_and_fylke(gold_table, kommune_table, fylke_table, srid)
df = joined_df.select(
    "pred_geom", "kommunenummer", "kommunenavn", "fylkesnummer", "fylkesnavn"
).withColumnRenamed("pred_geom", "geometry")
pdf = df.toPandas()
gdf = gpd.GeoDataFrame(pdf, geometry="geometry")
gdf.set_crs(epsg=25833, inplace=True)
gdf.to_file(local_gpkg, driver="GPKG")

# Kopierer fra lokalt til katalogen
os.makedirs(
    "/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/geopackages",
    exist_ok=True,
)
shutil.copyfile(local_gpkg, final_gpkg_path)

# Sletter lokal fil
os.remove(local_gpkg)